In [1]:
import re
import math

regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')

def isValid(email):
    if re.fullmatch(regex, email):
      return True
    else:
      print("Invalid email")
      return False

In [2]:
class Person:
    # class variable
    moods = ("happy", "tired", "Lazy")

    def __init__(self, name=None, money=None, mood=None, healthRate=None):
        self.name = name
        self.money = money
        self.mood = mood
        self._healthRate = healthRate

    @property
    def healthRate(self):
        return self._healthRate

    @healthRate.setter
    def healthRate(self, value):
        if value in {"100%", "75%", "50%"}:
            self._healthRate = value

    def sleep(self, hours=None):
        if hours == 7:
            self.mood = Person.moods[0]
        elif hours < 7:
            self.mood = Person.moods[1]
        else:
            self.mood = Person.moods[2]

    def eat(self, meals=None):
        health = {1: "100%", 2: "75%", 3: "50%"}
        self.healthRate = health[meals]

    def buy(self, items):
        self.money -= items*10

In [3]:
class Employee(Person):
    def __init__(self, name=None, money=None, mood=None, healthRate=None, id=None, car=None, email=None, salary=None, distanceToWork=20):
        super().__init__(name, money, mood, healthRate)
        self.id = id
        self.car = car
        self.email = email
        self.salary = salary
        self.distanceToWork = distanceToWork

    @property
    def salary(self):
        return self._salary

    @property
    def email(self):
        return self._email
    
    @salary.setter
    def salary(self, value):
        if value.isnumeric():
            if int(value) >= 1000:
                self._salary = value
            else:
                print("Enter a valid salary >= 1000.")
        else:
            print("Enter a valid salary as (integer).")

    @email.setter
    def email(self, value):
        if isValid(value):
            self._email = value
        else:
            print("Please Enter a valid email (xxxx@yyy.zzz)")
        
    def work(self, hours):
        if hours == 8:
            self.mood = Person.moods[0]
        elif hours > 8:
            self.mood = Person.moods[1]
        else:
            self.mood = Person.moods[2]

    def drive(self, distance):
        self.distanceToWork = distance
        velocity = distance/(9-7)
        print("Distance: {} km".format(self.distanceToWork), ", Velocity : {} km/h.".format(velocity))
        self.car.run(velocity, distance)

    def refuel(self, gasAmount=100):
        self.car.fuelRate += gasAmount

    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)

    def send_mail(self, to=None, subject=None, msg=None, reciever_name=None):
        pass

    def __str__(self):
        s = "#####################\n"
        i = "Emp ID: {}\n".format(self.id)
        n = "Emp Name: {}\n".format(self.name)
        c = "Emp Car: {}\n".format(self.car.name)
        ma = "Emp Email: {}\n".format(self.email)
        sa = "Emp Salary: {}\n".format(self.salary)
        mo = "Emp Account: {}\n".format(self.money)
        mood = "Emp Mood: {}\n".format(self.mood)
        hr = "Emp HealthRate: {}\n".format(self.healthRate)
        e = "#####################"
        #self.distanceToWork = distanceToWork
        return s + i + n + c + ma + sa + mo + mood + hr + e


In [4]:
class Office:
    employessNum = 0

    @staticmethod
    def calculate_lateness(moveHour, distance, velocity, targetHour=9):
        time = targetHour - moveHour
        calc_time = distance/velocity
        if calc_time <= time:
            #print("False")
            return False
        else:
            #print("True")
            return True

    def __init__(self, name=None):
        self.name = name
        self.employees = []

    def get_all_employees(self):
        return self.employees

    def get_employee(self, id):
        for emp in self.employees:
            if emp.id == id:
                return emp
            else:
                return "Not Found"

    def hire(self, employee):
        for e in self.employees:
            if e.id == employee.id:
                print("Employee is already exist")
                return False
        self.employees.append(employee)
        Office.employessNum += 1

    def fire(self, id):
        temp = self.get_employee(id)
        if temp:
            self.employees.remove(temp)
            Office.employessNum -= 1

    def check_lateness(self, empId, movHour):
        curEmp = self.get_employee(empId)
        flag = Office.calculate_lateness(moveHour=movHour, distance=curEmp.distanceToWork, velocity=curEmp.car.velocity)
        if flag:
            curEmp.salary = str(int(curEmp.salary) - 10)
        else:
            curEmp.salary = str(int(curEmp.salary) + 10)
            
    def deduct(self, empId, deduction):
        temp = self.get_employee(empId)
        temp.salary = str(int(temp.salary) - deduction)

    def reward(self, empId, reward):
        temp = self.get_employee(empId)
        temp.salary = str(int(temp.salary) + reward)

In [5]:
class Car:
    def __init__(self, name=None, fuelRate=None, velocity=None):
        self.name = name
        self.fuelRate = fuelRate
        self.velocity = velocity

    @property
    def fuelRate(self):
        return self._fuelRate

    @property
    def velocity(self):
        return self._velocity

    @fuelRate.setter
    def fuelRate(self, value):
        if value > -1 and value < 101:
            self._fuelRate = value
        else:
            print("Please Enter a valid rate (0 ~ 100).")

    @velocity.setter
    def velocity(self, value):
        if value > -1 and value < 201:
            self._velocity = value
        else:
            print("The Valid speed limit in (0 ~ 200)")

    def run(self, vel, dis):
        # -10% every 10km distance ---> -0.01/km, 1e-05/m
        # from km -> m
        dis_m = dis * 1000
        rate = 1e-5 * self.fuelRate
        flag = True
        self.velocity = vel
        while flag:
            if self.fuelRate <= 0 or dis_m == 0:
                flag = False
            else:
                self.fuelRate -= rate
                dis_m -= 1
        self.stop(dis_m)

    def stop(self, remainDist):
        self.velocity = 0
        print("Remaining Fuel: {}%".format(math.ceil(self.fuelRate)))
        if remainDist == 0:
            print("Arrived to the destination")
        else:
            print("The remaining distance {} km.".format(remainDist/1000))

In [6]:
car1 = Car(name="fiat128", fuelRate=100, velocity=200)
emp1 = Employee(salary="1500", name="Ahmed", money=10_000, id=1, email="ssss@gmail.com", car=car1)
print(emp1)

#####################
Emp ID: 1
Emp Name: Ahmed
Emp Car: fiat128
Emp Email: ssss@gmail.com
Emp Salary: 1500
Emp Account: 10000
Emp Mood: None
Emp HealthRate: None
#####################


In [7]:
emp1.sleep(6)
print(emp1)

#####################
Emp ID: 1
Emp Name: Ahmed
Emp Car: fiat128
Emp Email: ssss@gmail.com
Emp Salary: 1500
Emp Account: 10000
Emp Mood: tired
Emp HealthRate: None
#####################


In [8]:
emp1.work(8)
print(emp1)

#####################
Emp ID: 1
Emp Name: Ahmed
Emp Car: fiat128
Emp Email: ssss@gmail.com
Emp Salary: 1500
Emp Account: 10000
Emp Mood: happy
Emp HealthRate: None
#####################


In [9]:
emp1.eat(2)
print(emp1)

#####################
Emp ID: 1
Emp Name: Ahmed
Emp Car: fiat128
Emp Email: ssss@gmail.com
Emp Salary: 1500
Emp Account: 10000
Emp Mood: happy
Emp HealthRate: 75%
#####################


In [10]:
emp1.drive(110)

Distance: 110 km , Velocity : 55.0 km/h.
Remaining Fuel: 0%
The remaining distance 10.0 km.


In [11]:
emp1.car.fuelRate = 100
emp1.drive(50)

Distance: 50 km , Velocity : 25.0 km/h.
Remaining Fuel: 50%
Arrived to the destination


In [12]:
car1 = Car(name="fiat128", fuelRate=100, velocity=200)
emp1 = Employee(salary="1500", name="Ahmed", money=10_000, id=1, email="ssss@gmail.com", car=car1)

In [13]:
iti_office = Office(name="iti")

In [14]:
iti_office.hire(emp1)
print(Office.employessNum)

1


In [15]:
iti_office.hire(emp1)
print(Office.employessNum)

Employee is already exist
1


In [16]:
emp1.car.fuelRate = 100
emp1.car.velocity = 50
emp1.distanceToWork = 30
iti_office.check_lateness(1, 9)
print(iti_office.get_employee(1))

#####################
Emp ID: 1
Emp Name: Ahmed
Emp Car: fiat128
Emp Email: ssss@gmail.com
Emp Salary: 1490
Emp Account: 10000
Emp Mood: None
Emp HealthRate: None
#####################


In [17]:
iti_office.deduct(empId=1, deduction=40)
print(iti_office.get_employee(1))

#####################
Emp ID: 1
Emp Name: Ahmed
Emp Car: fiat128
Emp Email: ssss@gmail.com
Emp Salary: 1450
Emp Account: 10000
Emp Mood: None
Emp HealthRate: None
#####################


In [18]:
iti_office.reward(empId=1, reward=50)
print(iti_office.get_employee(1))

#####################
Emp ID: 1
Emp Name: Ahmed
Emp Car: fiat128
Emp Email: ssss@gmail.com
Emp Salary: 1500
Emp Account: 10000
Emp Mood: None
Emp HealthRate: None
#####################


In [19]:
iti_office.fire(1)
print(Office.employessNum)

0


In [20]:
def to_json(list_offices):
    final_result = {}
    for o in list_offices:
        list_emps = o.get_all_employees()
        emp_list_obj = []
        for e in list_emps:
            temp_emp = e.__str__().split("\n")[1:-1]
            emp_dict_obj = {}
            for t in temp_emp:
                tt = t.split(":")
                emp_dict_obj[tt[0]] = tt[1]
            emp_list_obj.append(emp_dict_obj)
        final_result[o.name] = emp_list_obj
    return final_result

In [21]:
car1 = Car(name="fiat128", fuelRate=100, velocity=200)
emp1 = Employee(salary="1500", name="Ahmed", money=10_000, id=1, email="ssss@gmail.com", car=car1)

car2 = Car(name="porsche", fuelRate=100, velocity=200)
emp2 = Employee(salary="20000", name="Nasser", money=10_000, id=2, email="mnasserce@gmail.com", car=car2)

car3 = Car(name="audi", fuelRate=100, velocity=200)
emp3 = Employee(salary="20000", name="Mostafa", money=10_000, id=3, email="malaa@gmail.com", car=car3)

iti = Office("iti")
iti.hire(emp1)
iti.hire(emp2)
iti.hire(emp3)

In [22]:
car1 = Car(name="fiat128", fuelRate=100, velocity=200)
emp1 = Employee(salary="1500", name="Ahmed", money=10_000, id=1, email="ssss@gmail.com", car=car1)

car2 = Car(name="porsche", fuelRate=100, velocity=200)
emp2 = Employee(salary="20000", name="Nasser", money=10_000, id=2, email="mnasserce@gmail.com", car=car2)

car3 = Car(name="audi", fuelRate=100, velocity=200)
emp3 = Employee(salary="20000", name="Mostafa", money=10_000, id=3, email="malaa@gmail.com", car=car3)

iti_2 = Office("iti_2")
iti_2.hire(emp1)
iti_2.hire(emp2)
iti_2.hire(emp3)

In [23]:
json_dict = to_json([iti, iti_2])

In [24]:
import json

with open("data.json", "w") as file:
    json.dump(json_dict, file)

In [25]:
to_json([iti, iti_2])

{'iti': [{'Emp ID': ' 1',
   'Emp Name': ' Ahmed',
   'Emp Car': ' fiat128',
   'Emp Email': ' ssss@gmail.com',
   'Emp Salary': ' 1500',
   'Emp Account': ' 10000',
   'Emp Mood': ' None',
   'Emp HealthRate': ' None'},
  {'Emp ID': ' 2',
   'Emp Name': ' Nasser',
   'Emp Car': ' porsche',
   'Emp Email': ' mnasserce@gmail.com',
   'Emp Salary': ' 20000',
   'Emp Account': ' 10000',
   'Emp Mood': ' None',
   'Emp HealthRate': ' None'},
  {'Emp ID': ' 3',
   'Emp Name': ' Mostafa',
   'Emp Car': ' audi',
   'Emp Email': ' malaa@gmail.com',
   'Emp Salary': ' 20000',
   'Emp Account': ' 10000',
   'Emp Mood': ' None',
   'Emp HealthRate': ' None'}],
 'iti_2': [{'Emp ID': ' 1',
   'Emp Name': ' Ahmed',
   'Emp Car': ' fiat128',
   'Emp Email': ' ssss@gmail.com',
   'Emp Salary': ' 1500',
   'Emp Account': ' 10000',
   'Emp Mood': ' None',
   'Emp HealthRate': ' None'},
  {'Emp ID': ' 2',
   'Emp Name': ' Nasser',
   'Emp Car': ' porsche',
   'Emp Email': ' mnasserce@gmail.com',
   'Emp 

In [26]:
import json

x = iti.get_all_employees()
for e in x:
    print(e.toJSON())

{
    "_email": "ssss@gmail.com",
    "_healthRate": null,
    "_salary": "1500",
    "car": {
        "_fuelRate": 100,
        "_velocity": 200,
        "name": "fiat128"
    },
    "distanceToWork": 20,
    "id": 1,
    "money": 10000,
    "mood": null,
    "name": "Ahmed"
}
{
    "_email": "mnasserce@gmail.com",
    "_healthRate": null,
    "_salary": "20000",
    "car": {
        "_fuelRate": 100,
        "_velocity": 200,
        "name": "porsche"
    },
    "distanceToWork": 20,
    "id": 2,
    "money": 10000,
    "mood": null,
    "name": "Nasser"
}
{
    "_email": "malaa@gmail.com",
    "_healthRate": null,
    "_salary": "20000",
    "car": {
        "_fuelRate": 100,
        "_velocity": 200,
        "name": "audi"
    },
    "distanceToWork": 20,
    "id": 3,
    "money": 10000,
    "mood": null,
    "name": "Mostafa"
}


In [27]:
emp3.sleep(8)